In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 10)
pd.options.display.float_format = '{:.3f}'.format
import warnings; warnings.filterwarnings('ignore')

# Import Data & Preliminary Transformations

In [2]:

old_train = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/train.csv')
new_train = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting_new/revealed_test.csv')
old_test = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/test.csv')
sample_submission = pd.read_csv('./data/raw/godaddy-microbusiness-density-forecasting/sample_submission.csv')

train = pd.concat((old_train, new_train))
test = old_test[~old_test['first_day_of_month'].isin(new_train['first_day_of_month'])]

train['is_test'] = 0 ; test['is_test'] = 1

data = pd.concat((
        train,
        test)
        )\
    .reset_index(drop=True)\
    .assign(
        cfips = lambda df: df['cfips'].astype(str).str.zfill(5),
        date = lambda df: pd.to_datetime(df["first_day_of_month"]),
        )\
    .sort_values(['cfips','date'], ascending=True)\
    .assign(
    
        state_i = lambda df: df['cfips'].apply(lambda x: x[:2]),
        county_i = lambda df: df['cfips'].apply(lambda x: x[2:]),        

        year = lambda df: df['date'].dt.year,
        date = lambda df: df["date"].dt.date,

        period = lambda df: df.groupby('cfips')['row_id'].cumcount(),

        active_lag_1 = lambda df: df.groupby('cfips')['active'].shift(1),
        active_lag_2 = lambda df: df.groupby('cfips')['active'].shift(2),
        active_lag_3 = lambda df: df.groupby('cfips')['active'].shift(3),
        active_lag_4 = lambda df: df.groupby('cfips')['active'].shift(4),

        target_0 = lambda df: np.nan_to_num(df['active']/df.groupby('cfips')['active'].shift(1)-1, posinf=10),
        target_1 = lambda df: np.nan_to_num(df['active']/df.groupby('cfips')['active'].shift(2)-1, posinf=10),
        target_2 = lambda df: np.nan_to_num(df['active']/df.groupby('cfips')['active'].shift(3)-1, posinf=10),

    )\
    .drop(['county','state'], axis='columns')
    
data.loc[data.is_test==1,['target_0','target_1','target_2']]  = np.nan

assert all(data.groupby('cfips')['county_i'].nunique() == 1)
assert all(data.groupby('cfips')['state_i'].nunique() == 1)
assert data['cfips'].nunique() == 3135 # there are 3135 county,state tuples
assert data['period'].nunique() == 47 # there are 47 series for each county state tuple
assert data.query('is_test==0')['period'].nunique() == 41 # there are 41 series in the train set. 
assert data.query('is_test==1')['period'].nunique() == 6  # there are 6 series in the test set. 

# the puclic lb is updated as 01-2023
# the private lb will include 03-2023, 04-2023, 05-2023
# we dont care about 06-2023

In [3]:
data.query("cfips == '01001'").tail(10)

,row_id,cfips,first_day_of_month,microbusiness_density,active,is_test,date,state_i,county_i,year,period,active_lag_1,active_lag_2,active_lag_3,active_lag_4,target_0,target_1,target_2
37,1001_2022-09-01,01001,2022-09-01,3.443,1463.000,0,2022-09-01,01,001,2022,37,1455.000,1461.000,1422.000,1408.000,0.005,0.001,0.029
38,1001_2022-10-01,01001,2022-10-01,3.464,1472.000,0,2022-10-01,01,001,2022,38,1463.000,1455.000,1461.000,1422.000,0.006,0.012,0.008
122265,1001_2022-11-01,01001,2022-11-01,3.443,1463.000,0,2022-11-01,01,001,2022,39,1472.000,1463.000,1455.000,1461.000,-0.006,0.000,0.005
122266,1001_2022-12-01,01001,2022-12-01,3.471,1475.000,0,2022-12-01,01,001,2022,40,1463.000,1472.000,1463.000,1455.000,0.008,0.002,0.008
128535,1001_2023-01-01,01001,2023-01-01,NaN,NaN,1,2023-01-01,01,001,2023,41,1475.000,1463.000,1472.000,1463.000,NaN,NaN,NaN
131670,1001_2023-02-01,01001,2023-02-01,NaN,NaN,1,2023-02-01,01,001,2023,42,NaN,1475.000,1463.000,1472.000,NaN,NaN,NaN
134805,1001_2023-03-01,01001,2023-03-01,NaN,NaN,1,2023-03-01,01,001,2023,43,NaN,NaN,1475.000,1463.000,NaN,NaN,NaN
137940,1001_2023-04-01,01001,2023-04-01,NaN,NaN,1,2023-04-01,01,001,2023,44,NaN,NaN,NaN,1475.000,NaN,NaN,NaN
141075,1001_2023-05-01,01001,2023-05-01,NaN,NaN,1,2023-05-01,01,001,2023,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144210,1001_2023-06-01,01001,2023-06-01,NaN,NaN,1,2023-06-01,01,001,2023,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# adding census data
data_census = []
for year in range(2017,2022):
    COLS = ['GEO_ID','NAME','S0101_C01_026E']
    data_census_i = pd.read_csv(f'./data/raw/census_data_1/ACSST5Y{year}.S0101-Data.csv',usecols=COLS)
    data_census_i = data_census_i.iloc[1:]
    data_census_i['population'] = data_census_i['S0101_C01_026E'].astype('int')


    data_census_i['cfips'] = data_census_i.GEO_ID.apply(lambda x: f"{int(x.split('US')[-1]):05}" )
    data_census_i['year'] = year+2
    data_census.append(data_census_i[['cfips','year','population']])

data_census = pd.concat((data_census),axis='rows')
data = data.merge(data_census, on=['cfips','year'], how='left')

## Parameters


In [5]:
# parameters for cv
n_SPLITS = 5 
n_TRAIN_TRAIN_SIZE = 6
n_TRAIN_PERIOD = n_TRAIN_TRAIN_SIZE + 3 + n_SPLITS - 1 

# parameters used for filtering/selecting data
TEST_DATES = list(np.sort(data.query('is_test==1')['date'].unique())[:3])
TEST_PERIOD = list(np.sort(data.query('is_test==1')['period'].unique())[:3])

TRAIN_PERIOD = list(np.sort(data.query('is_test==0')['period'].unique())[-n_TRAIN_PERIOD:])
TRAIN_DATES = list(np.sort(data.query('is_test==0')['date'].unique())[-n_TRAIN_PERIOD:])

# some parameters which we will use for feature engineering
LEAKAGE = ['microbusiness_density','active']
TARGETS = ['target_0', 'target_1', 'target_2']


## Sample Data

In [6]:
# here we sample the data since there is no need to use all the data for training according to our cv parameters
# also we prepare the lag values here since later we will devide the train/test data to avoid any leakage during the feature engineering

sample = data.copy()
sample = sample.sort_values(['cfips','date'])

LAGS = 4
for i in range(1, LAGS+1):
    lag_col = f'target_lag{i}'
    sample[lag_col] = sample.groupby('cfips')[TARGETS[0]].shift(i)  

# need to sort values by period for GroupTimeSeriesSplit
sample = sample.sort_values('period')

sample_train= sample.query("period in @TRAIN_PERIOD") ; sample_test= sample.query("period in @TEST_PERIOD")
train_X = sample_train.drop(TARGETS,axis='columns') ; train_y = sample_train[TARGETS]
test_X = sample_test.drop(TARGETS,axis='columns') ; test_y = sample_test[TARGETS]


# Pipelining

In [7]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(transform_output="pandas")
from sklearn.metrics import make_scorer
from mlxtend.evaluate.time_series import GroupTimeSeriesSplit
from collections import defaultdict

def SMAPE (y_true, y_pred):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true = np.array(y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns"""
    def __init__(self, features):
        self.features = features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.features]
    
class FeatureEngineering(BaseEstimator, TransformerMixin):
    """Make some feature engineering on selected columns"""
    def __init__(self, features):
        self.features = features
	    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        new_features = []
        return X_transformed[new_features]
    


lag=2
list_lag_values = [[f'target_lag{lag_i+model_i+1}' for lag_i in range(lag)] for model_i in range(3)]

dic_pipelines = {}
y_test_preds  = [] 
y_validation_preds = defaultdict(list)
validation_scores = defaultdict(list)


for model_i in range(3):
    # chose the right target to model 
    train_y_i = train_y.iloc[:, model_i]
    
    # raw features without extra feature engineering
    raw_features = Pipeline([('select features', ColumnSelector(features=list_lag_values[model_i]))])
    # features after some transformation (in this case there is no transformed feature)
    new_features = Pipeline([('engineer selected features', FeatureEngineering(features=[]))])
     
    merge_features = FeatureUnion([
        ('raw_features', raw_features),
        ('new_features', new_features)
    ])

    # last transformation to all numeric features such as removing outliers (here no transformation is made)
    numeric_features = Pipeline([
                            ('merge_features',merge_features),
                            # ('remove_outliers', Winsorizer()),
                            # ('standart_scaler', StandardScaler())
                            ]
                            )
    # other type of transformations can be made to different type of fields such as categorical features and added to the pipeline later
    # categrical_features = Pipeline([])

    # transformedTargetRegressor helps to make target transformation such as log within the pipeline
    model = TransformedTargetRegressor()  
    
    # model pipeline = feature + model 
    model_pipeline = Pipeline([
        ("transform_numeric", numeric_features),
        # ("transform_categorical", categorical_features),
        ("model", model)
    ])

    
    # parameters to tune the model should be added here
    # param_grid = {'model__regressor':[LinearRegression(),LinearSVR()]}
    param_grid = {'model__regressor':[LinearRegression()]}
    
    cv_args = {"test_size": 1, "n_splits": n_SPLITS, "train_size": n_TRAIN_TRAIN_SIZE, 'gap_size': 0}
    cv = GroupTimeSeriesSplit(**cv_args)
    
    grid = GridSearchCV(model_pipeline, scoring=make_scorer(SMAPE, greater_is_better=False), param_grid=param_grid, cv=cv)
    
    grid.fit(train_X, train_y_i, groups=train_X['period'])
    
    # pipeline can be persisted at a dictionary
    dic_pipelines[f'pipeline_model_{model_i}'] = grid

    # Validation Scores
    # since we use the ratio of active's as the target value the error metric is not comparable to what the competition calculates
    # so here we calculate the validation score for the selected model by using absolute values of active
    # the error should be used to sleect which model to use as final submission
    # otherwise there is a risk of overfitting
    # we also store these values to make further error analysis

    train_period = TRAIN_PERIOD[-1-n_TRAIN_TRAIN_SIZE: -1] 
    validation_period = [TRAIN_PERIOD[-1]]
    train_index = train_X.query('period in @train_period').index
    val_index = train_X.query('period in @validation_period').index

    best_model = grid.best_estimator_
    best_model.fit(train_X.loc[train_index], train_y_i.loc[train_index])  
    
    y_val_pred =  (best_model.predict(train_X.loc[val_index])+1)*train_X.loc[val_index,f'active_lag_{model_i+1}']	
    y_validation_preds[f'target_{model_i}'] = y_val_pred
    
    validation_scores[f'error_{model_i}'] = SMAPE(y_true= train_X.loc[val_index,'active'], y_pred=y_val_pred)

    # Inference
    inference_train_period = TRAIN_PERIOD[-n_TRAIN_TRAIN_SIZE:] 
    inference_test_period_i = [TEST_PERIOD[model_i]]
    inference_train_index = train_X.query('period in @inference_train_period').index

    best_model.fit(train_X.loc[inference_train_index], train_y_i.loc[inference_train_index])
    y_test_preds.append(best_model.predict(test_X.query('period in @inference_test_period_i')))


In [8]:
# we can see some details about the pipelines that we used to model
dic_pipelines['pipeline_model_0'].best_estimator_

Pipeline(steps=[('transform_numeric',
                 Pipeline(steps=[('merge_features',
                                  FeatureUnion(transformer_list=[('raw_features',
                                                                  Pipeline(steps=[('select '
                                                                                   'features',
                                                                                   ColumnSelector(features=['target_lag1',
                                                                                                            'target_lag2']))])),
                                                                 ('new_features',
                                                                  Pipeline(steps=[('engineer '
                                                                                   'selected '
                                                                                   'features',
                                                                                   FeatureEngineering(features=[]))]))]))])),
                ('model',
                 TransformedTargetRegressor(regressor=LinearRegression()))])

In [9]:

# prepare validation for error analysis
# val_X = train_X.query('dcount in @check_validation_period')
# y_val_preds =  pd.DataFrame(y_validation_preds, index=val_X.index)
# val_X = pd.concat((val_X, y_val_preds), axis=1)

# this is the validation score which we use to have a feeling about the model performance on the test set
validation_scores

defaultdict(list,
            {'error_0': 1.8583042562020717,
             'error_1': 2.4796749903226614,
             'error_2': 2.9173206825002134})

In [10]:
# we start to prepare the predictions for submission
test_X['ratio_pred'] = np.concatenate((y_test_preds))

for i,TEST_PERIOD_i in enumerate(TEST_PERIOD):
    test_index = test_X.query('period == @TEST_PERIOD_i').index 
    test_X.loc[test_index,'pred'] = (test_X.loc[test_index]['ratio_pred']+1)*test_X.loc[test_index][f'active_lag_{i+1}']

# we replace the predictions for low populated zones with their lag values (benchmark values). 
# the rule for population is lower than .3 quantile
test_X['final_pred'] = test_X['pred']
condition = test_X['population']<np.quantile(test_X['population'],q=.3)

# we can change the rule for the benchmark values such as mean of last 2 lag values etc
test_X = test_X.sort_values(['cfips','first_day_of_month'])
test_X['benchmark'] = test_X.groupby('cfips').first()[['active_lag_1','active_lag_1','active_lag_1']].stack().values

test_X.loc[condition,'final_pred'] = test_X.loc[condition,'benchmark']

In [11]:
# the submission file


date_submission = '1003'
local_score = round(validation_scores['error_0'],2)
model_name = 'ratio_regression_lag_1_2_with_.3_constant'

df_output = test_X.assign(
    microbusiness_density = lambda df: 100 * df['final_pred'] / df['population'],
    row_id = lambda df: df.apply(lambda df: "{}_{}".format(int(df['cfips']),df['date']), axis='columns'))[['row_id','microbusiness_density']]

submission = pd.concat((
    df_output,
    sample_submission[~sample_submission.row_id.isin(df_output.row_id)]))

# submission.to_csv(f"data/{date_submission}_{model_name}_local_{local_score}.csv",index=None)


In [12]:
submission

,row_id,microbusiness_density
41,1001_2023-01-01,3.343
42,1001_2023-02-01,3.358
43,1001_2023-03-01,3.370
88,1003_2023-01-01,7.994
89,1003_2023-02-01,8.035
...,...,...
25075,56037_2023-06-01,3.818
25076,56039_2023-06-01,3.818
25077,56041_2023-06-01,3.818
25078,56043_2023-06-01,3.818
